In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from matplotlib import patches
from matplotlib.collections import PatchCollection

import os
# from pandas.io.json import json_normalize
import pandas as pd

In [1]:
from src/dfLoader import *


SyntaxError: invalid syntax (<ipython-input-1-e8f30ae4feed>, line 1)

In [2]:
audio_dir = 'example_audio/bat_data_martyn_2018/'
ann_path = 'annotations/BritishBatCalls_MartynCooke_2018_1_sec_train_expert.json'
path = 'annotations/'

## Helper functions

In [4]:
def gen_mag_spectrogram(x, nfft, noverlap):
    # Computes magnitude spectrogram by specifying time.

    # window data
    step = nfft - noverlap
    shape = (nfft, (x.shape[-1]-noverlap)//step)
    strides = (x.strides[0], step*x.strides[0])
    x_wins = np.lib.stride_tricks.as_strided(x, shape=shape, strides=strides)

    # apply window
    x_wins = np.hanning(x_wins.shape[0]).astype(np.float32)[..., np.newaxis] * x_wins

    # do fft
    # note this will be much slower if x_wins.shape[0] is not a power of 2
    complex_spec = np.fft.rfft(x_wins, axis=0)

    # calculate magnitude
    #spec = (np.conjugate(complex_spec) * complex_spec).real
    # same as:
    spec = np.absolute(complex_spec)**2

    # orientate the ocrrect way 
    spec = np.flipud(spec[W 19:57:07.859 NotebookApp] Notebook LoadData.ipynb is not trusted
)
  
    # convert to "amplitude"
    spec = np.log(1.0 + spec)

    return spec

def plot_bounding_box_patch(ann, freq_scale, colors):
    patch_collect = []
    for aa in ann['annotation']:
        xx = aa['start_time']
        ww = aa['end_time'] - aa['start_time']
        yy = aa['low_freq'] / freq_scale
        hh = (aa['high_freq'] - aa['low_freq']) / freq_scale
        col = colors[int(aa['individual'])]
        patch_collect.append(patches.Rectangle((xx, yy), ww, hh, linewidth=1,
                             edgecolor=col, facecolor='none', alpha=1.0))
    return patch_collect

SyntaxError: invalid syntax (<ipython-input-4-88b0692753a9>, line 23)

 # Load All Audio FIle

In [3]:
for file in os.listdir(path):
    print(file)

BritishBatCalls_MartynCooke_2019_1_sec_train_expert.json
Echobank_train_expert.json
bcireland_expert.json
sn_scot_nor_0.5_expert.json
BritishBatCalls_MartynCooke_2019_1_sec_test_expert.json
BritishBatCalls_MartynCooke_2018_1_sec_train_expert.json
BritishBatCalls_MartynCooke_2018_1_sec_test_expert.json
BCT_1_sec_train_expert.json


In [4]:
for file in os.listdir('audio'):
    print(file)

bat_data_martyn_2019
sn_scot_nor
echobank
bat_data_martyn_2018_test
BCT_1_sec
bat_audio_april_2021.zip
bat_data_martyn_2019_test
bcireland
bat_data_martyn_2018


In [5]:
data = json.load(open(ann_path))

df = pd.json_normalize(data, record_path=['annotation'], meta=['id'])

In [6]:
duration = df['end_time'] - df['start_time']

df['duration'] = duration

trimInTime = []
trimOutTime = []

for index, row in df.iterrows():
    startTime = row['start_time']
    endTime   = row['end_time']
    middleTime = (endTime + startTime) / 2
    if(middleTime - 0.01 > 0):
        trimInTime.append(middleTime - 0.01)
        trimOutTime.append(middleTime + 0.01)
    else:
        trimInTime.append(0)
        trimOutTime.append(0.02)
        
df['trimInTime'] = trimInTime
df['trimOutTime'] = trimOutTime

df.describe()

,end_time,high_freq,low_freq,start_time,duration,trimInTime,trimOutTime
count,6242.000000,6242.000000,6242.000000,6242.000000,6242.000000,6242.000000,6242.000000
mean,0.496609,82591.596853,44101.257813,0.484339,0.012270,0.480497,0.500497
std,0.285968,25129.398756,20204.163371,0.285923,0.015195,0.285806,0.285806
min,0.006974,21748.535156,15625.000000,0.001799,0.001805,0.000000,0.020000
25%,0.251055,61523.437500,28320.312500,0.238316,0.004937,0.235054,0.255054
50%,0.493318,84000.000000,35644.531250,0.480982,0.006172,0.477763,0.497763
75%,0.742767,104478.149414,53710.937500,0.729807,0.008641,0.726078,0.746078
max,0.999875,150666.666667,147333.333333,0.996531,0.102865,0.988177,1.008177


In [7]:
trimmed_data = []

for filename in df['id'].unique():
    fs, x = wavfile.read(audio_dir + filename)
    fileDuration = x.shape[0]/fs
    spec = gen_mag_spectrogram(x, 1024, 768)
    
    for index, row in df[df['id'] == filename].iterrows():
        startNum = int(row['trimInTime']/fileDuration * spec.shape[0])
        endNum   = int(row['trimOutTime']/fileDuration * spec.shape[0])
        
        trimmed_data.append(spec[0:spec.shape[0],startNum:endNum])
    
    


FileNotFoundError: [Errno 2] No such file or directory: 'example_audio/bat_data_martyn_2018/BARBAR_20180723_213212-BARBAR-LR_0_1.0.wav'

In [8]:
df['data'] = trimmed_data

SyntaxError: unmatched '}' (70646322.py, line 1)